<a href="https://colab.research.google.com/github/marcosbenicio/deep_learning_classification/blob/main/classificador_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificador para os dígitos $0$ e $5$

Instalação manual de pacote

In [527]:
!pip install -Uqq fastbook

Biblioteca fastai permite usar funções para rapidamente construir uma rede neuronal e treinar nosso modelo de diferenciar $0$ e $5$.

In [528]:
import fastbook
from fastai.vision.all import *
fastbook.setup_book()
from fastbook import *
import numpy as np

# Muda a escala de cor padrão da imagem para escala de cinza.
matplotlib.rc('image', cmap='Greys')

import PIL as pl
import torch as t
import pandas as pd

Primeiro passamos o caminho do diretório dataset MINST para a variável path. Nesse diretório temos imagens para todos os dígitos de $0$ à $9$. Em seguida, com o código (/path/...).ls( ), é possível listar o conteúdo dentro desse diretório. Temos portanto a pasta 'training' e 'testing', e dentro de cada uma delas mais outros $10$ diretórios referentes aos números de $0$ à $9$. Esse método retorna uma lista contendo os caminhos dos arquivos de imagem, cada posição da lista têm armazenado o caminho para um arquivo de imagem.

Por último, nessa célula, vamos criar quatro listas com os caminhos dos arquivos em ordem crescente. Teremos duas listas contendo todos os caminhos das imagens de $0$s(zeros) presentes nos diretórios 'training' e 'testing', e outros duas listas para as imagens de $5$s(cincos). 

In [529]:
# Armazena o caminho para o diretório do dataset MNIST numa variável.
path = untar_data(URLs.MNIST)

# Para o diretório 'training' 
zeros_train = (path/'training'/'0').ls().sorted()
cincos_train = (path/'training'/'5').ls().sorted()

# Para o diretório 'testing' 
zeros_test = (path/'testing'/'0').ls().sorted()
cincos_test = (path/'testing'/'5').ls().sorted()

Criarei quatro listas contendo tensores de segunda ordem (matrizes)  de dimensão $28\times28$ representando cada uma das imagens. Portanto ficarei com duas listas contendo todas as imagens de $0$ dos diretórios 'training' e 'testing', e outras duas listas para as imagens de $5$. 

Além disso, vamos empacotar cada lista num tensor de primeira ordem (vetor) usando o método stack da biblioteca Pytorch. Irei também normalizar o tensor para os elementos do último eixo assumirem valores entre $0$ e $1$.
Para o diretório de zeros, por exemplo, o tensor que definiremos como zeros_train_tensor terá $5923$ matrizes de dimensão $28\times28$.

 

In [530]:
# Para o diretório 'training'
zeros_train_list = [tensor(Image.open(i)) for i in zeros_train]
cincos_train_list = [tensor(Image.open(i)) for i in cincos_train]

zeros_train_tensor = t.stack(zeros_train_list).float()/255
cincos_train_tensor = t.stack(cincos_train_list).float()/255

# Para o diretório 'testing'
zeros_test_list = [tensor(Image.open(i)) for i in zeros_test]
cincos_test_list = [tensor(Image.open(i)) for i in cincos_test]

zeros_test_tensor = t.stack(zeros_test_list).float()/255
cincos_test_tensor = t.stack(cincos_test_list).float()/255


Nossa variável independente $x$ serão as imagens $0$s e $5$s do diretório 'training' concatenanas num único tensor x_train. Para isso usarei o método view, que  muda o formato do tensor sem alterar seu conteúdo. O $-1$ é um parâmetro especial que pega o tamanho necessário de um eixo para caber todos os dados.

O tensor y_train armazenará o rótulo referente a cada imagem do tensor x_train. O método unsqueeze(1) adiciona uma dimensão ao tensor, ficando equiparável ao rank do tensor x_train. O mesmo processo vai ser feito também para o diretório 'testing', que chamarei de x_test e y_test. A isso chamamos de Dataset.

Além disso, usando a função zip podemos colocar elementos de mesmo índice e diferentes tensores como elementos de uma mesma tupla (x,y). Portanto, relacionariamos cada imagem de x_test com um rótulo de y_test através das tuplas (x,y) como um conjunto de pontos de um gráfico. Portanto as variáveis definidas como dset_train e dset_test chamamos de Dataset.


In [531]:
# Para o diretório 'training'
x_train = t.cat([zeros_train_tensor, cincos_train_tensor]).view(-1, 28*28)
y_train = tensor([0]*len(zeros_train) + [1]*len(cincos_train)).unsqueeze(1)

dset_train = list(zip(x_train, y_train))

# Para o diretório 'testing'
x_test = t.cat([zeros_test_tensor, cincos_test_tensor]).view(-1, 28*28)
y_test = tensor([0]*len(zeros_test) + [1]*len(cincos_test)).unsqueeze(1)

dset_test = list(zip(x_test, y_test))

Enquanto o Dataset armazena todos os dados de interesse em tuplas, o Dataloader é uma classe usado para iterar esses dados, gerenciar batches, e transformar dados. No nosso caso temos dois Datasets, dset_train e dset_test. Cada um deles armazenam tuplas das variáveis dependentes (rótulos $0$ e $1$ para as imagens de $0$s e de $5$s) e independes ( imagens de $0$s e $5$s representadas por tensores). Passaremos para a classe Dataloader  os Dataset para criar os objetos dl_train e dl_test.

In [532]:
dl_train = DataLoader(dset_train, batch_size=256, suffle=False)
dl_test = DataLoader(dset_test, batch_size=256, suffle=False)

Vamos inicializar um tensor usando o método randn(size) dentro da função inicia_parametros( ). Essa função recebe o número total de pixels presentes nas imagens, que no nosso caso são $28 \times 28=784$ pixels.
Usando essa função teremos 4 tensores para inicializar, 2 weights (peso) e 2 bias nomeados de w1, w2, b1, b2.



In [533]:
def inicia_parametros(num_pixel): return (t.randn(num_pixel)).requires_grad_()

w1 = inicia_parametros((28*28,30))
b1 = inicia_parametros(30)
w2 = inicia_parametros((30,1))
b2 = inicia_parametros(1)


Definimos nossa função rede_simples responsável por melhor aproximar nossos parâmetros. Ela é uma função não-linear, tendo em vista que ela é composta por equações lineares e uma não linear acoplada às demais. Usamos como função max() para acresentar essa não-linearidade da função.

Dado a possível incapacidade de da GPU  processar todo nosso x_train de uma única vez pela função rede_simples, iremos separar um batch, como se fosse um pacote menor de dados que será processado por vez.

Além disso, é definida a função mnist_loss, responsável por medir a média da distância dentre o resultado previsto e o resultado esperado do batch. Queremos verificar o quão bem conseguimos prever nossa variável dependente a partir da variável independente com uso da rede_simples.

In [534]:
def rede_simples(xb): 
    res = xb@w1 + b1    
    res = res.max(t.tensor(0.0))
    res = res@w2 + b2
    return res

batch = x_train[:4]
y_train_prev = rede_simples(batch)

def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()

loss = mnist_loss(y_train_prev, y_train[:4])

Nessa etapa definimos a função grad para calcular a derivada da função que definimos como rede_simples em cada ponto (x,y).


In [535]:
def grad(xb,yb, model ):
  preds = model(xb)
  loss = mnist_loss(preds, yb)
  loss.backward()

grad(x_train[:30], y_train[:30], rede_simples)

Definimos uma função train_epoch para atualizar os parâmetros w1,w2,b1,b2. Criamos dois loops, onde no primeiro recebemos tuplas do tipo (xb,yb), retornadas pelo objeto iterativo dl_train da classe DataLoader, que são passadas para função grad para o cálculo da derivada da função rede_simples no ponto (xb,yb). No segundo loop, atualizamos os parâmetros e zeramos as derivadas para guardarmos na memória apenas a de interesse naquele instante do loop.




In [536]:
def train_epoch(model, lr, params):
    for xb,yb in dl_train:
        grad(xb, yb, model)
        for p in params:
            p.data -= p.grad*lr  #decremento (taxa de aprendizado de maquina?)
            p.grad.zero_()

Vamos definir a função que vai comparar nossos resultados, nos retornando a precisão de um batch. A função sigmoid é usada para normalizar todos os valores da xb que essa função recebe, para que em seguida, possamos comparar com a variável dependente yb, criando um tensor booleano contendo False ou True. Em seguida, é retornada a média desse tensor. Essa será a precisão do modelo para um batch(=x_train[0:4]). 

Para englobar todos os batchs devemos portanto definir a função validate_epoch. Ela consiste de um loop que busca tuplas (xb,yb) do objeto Dataloader dl_test e coloca como input da função batch_accuracy. Portanto, teremos agora a precisão mais robusta do modelo para todas as batchs com essa função.

In [537]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()  
    correct = (preds>0.5) == yb
    return correct.float().mean()
    
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb,yb in dl_test]
    return round(t.stack(accs).mean().item(), 4)

Por fim, podemos chamar as variáveis que usaremos e rodar o modelo. O processo de treino pode ser então observado, onde com 20 epochs obtive precisão de 98% quando comparados as imagens de 0s e 5s do diretório training com o diretório testing.

In [538]:
lr = 1
params = w1,w2,b1,b2
epochs = 20
validate_epoch(rede_simples)
for i in range(epochs):
    train_epoch(rede_simples, lr, params)
    print(validate_epoch(rede_simples), end=' ')

0.5371 0.6123 0.7587 0.804 0.8577 0.882 0.904 0.9079 0.9299 0.9455 0.9563 0.9626 0.966 0.9688 0.9737 0.9747 0.9752 0.9786 0.9791 0.9801 